---
title: "<u>Customer Segmentation with PCA & KMeans</u>"
image: "/images/segmentation.jpg"
format:
  html:
    toc: true
    code-fold: false
execute:
  echo: true
  eval: true
  output: true
  engine : jupyter
  jupyter: python3
---

## Project Overview

Performed customer segmentation by applying PCA for dimensionality reduction followed by KMeans clustering. Explored cluster counts from 2 to 6 and evaluated the results using both the Elbow Method and silhouette scores to ensure well-separated groupings. The analysis uncovered distinct clusters characterized by differences in shopping habits, demographics, and financial behavior, providing actionable insights for targeted marketing strategies.

## My Contributions

-   Performed data cleaning and exploratory analysis.
-   Executed PCA for dimensionality reduction.
-   Built KMeans clustering models.
-   Visualized clustering outcomes and interpreted customer segments.

## Technologies Used

-   Python (Pandas, NumPy, Scikit-learn, Matplotlib, Seaborn)
-   Jupyter Notebook

## Code

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.cluster import KMeans
import numpy as np
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score

In [ ]:
file_path = "Segmented_Data_PCA_KMeans.xlsx"
df = pd.read_excel(file_path)

In [ ]:
segmentation_vars = ["Age", "Gender", "Shopping_on_Amazon", "annual_income_US", "Parent", "Credit"]
df_segment = df[segmentation_vars]

In [ ]:
label_encoders = {}
categorical_vars = ["Gender", "Shopping_on_Amazon", "Parent", "Credit"]

for col in categorical_vars:
    le = LabelEncoder()
    df_segment[col] = le.fit_transform(df_segment[col].astype(str))
    label_encoders[col] = le

In [ ]:
df_segment["annual_income_US"].fillna(df_segment["annual_income_US"].median(), inplace=True)

In [ ]:
df_segment["annual_income_US"].fillna(df_segment["annual_income_US"].median(), inplace=True)

In [ ]:
print(df_segment.isna().sum())

pca = PCA(n_components=2) 
df_pca = pca.fit_transform(df_segment) 
df_pca = pd.DataFrame(df_pca, columns=["PC1", "PC2"]) 
print(df_pca)

In [ ]:
wcss = []
silhouette_scores = []

for k in range(2, 7): 
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(df_pca)
    wcss.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(df_pca, labels))
    
plt.figure(figsize=(8, 5))
plt.plot(range(2, 7), wcss, marker="o")
plt.xlabel("Number of Clusters (K)")
plt.ylabel("WCSS")
plt.title("Elbow Method for Optimal K")
plt.show()

for i, k in enumerate(range(2, 7)):
    print(f"K={k}, Silhouette Score={silhouette_scores[i]:.4f}")

In [ ]:
optimal_k = 3 
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df["Segment"] = kmeans.fit_predict(df_pca)
df.to_excel("Segmented_Data_PCA_KMeans.xlsx", index=False)

In [ ]:
plt.figure(figsize=(8, 6))
sns.scatterplot(x=df_pca["PC1"], y=df_pca["PC2"], hue=df["Segment"], palette="viridis")
plt.title(" K=3")
plt.xlabel("Principal Component 1")
plt.ylabel("Principal Component 2")
plt.legend(title="Cluster")
plt.show()

## Output

In [ ]:
print(df.groupby("Segment").mean())

In [ ]:
segment_summary = df.groupby("Segment").mean()

from IPython.display import display
display(segment_summary)

In [ ]:
segment_counts = df["Segment"].value_counts()
total_count = len(df)

segment_percentage = (segment_counts / total_count) * 100
segment_percentage_df = segment_percentage.reset_index()
segment_percentage_df.columns = ["Segment", "Percentage"]
display(segment_percentage_df)

In [ ]:
percent_by_gender = pd.crosstab(
    df["Gender"], 
    df["Segment"], 
    normalize="index"
) * 100

print("Row-Wise ")
display(percent_by_gender)
percent_by_segment = pd.crosstab(
    df["Gender"], 
    df["Segment"], 
    normalize="columns"
) * 100

print("Column-Wise ")
display(percent_by_segment)
counts_with_totals = pd.crosstab(
    df["Gender"], 
    df["Segment"], 
    margins=True
)
display(counts_with_totals)